<a href="https://colab.research.google.com/github/asmundur/deeplearning-finalproject/blob/master/Copy_of_deep_learning_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#code based on kernel https://www.kaggle.com/hrmello/flow-from-dataframe-a-memory-friendly-approach


from googleapiclient.discovery import build
import io, os, cv2, csv
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow
from skimage.io import imread

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Concatenate
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [7]:
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)



!pip install kaggle


Download 100%.


In [0]:
# if not os.path.exists('./data'):
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
if (not os.path.exists('./train')) and (not os.path.exists('./test')):
  !kaggle competitions download -c humpback-whale-identification
  !unzip -q train.zip -d train
  !unzip -q test.zip -d test
  !ls train | wc -l
  !ls test | wc -l
  


  0% 0.00/498k [00:00<?, ?B/s]
100% 498k/498k [00:00<00:00, 33.3MB/s]
  0% 0.00/594k [00:00<?, ?B/s]
100% 594k/594k [00:00<00:00, 38.9MB/s]
 99% 1.34G/1.35G [00:21<00:00, 73.0MB/s]
100% 1.35G/1.35G [00:21<00:00, 67.3MB/s]
 15% 640M/4.16G [00:13<01:01, 61.5MB/s]

In [0]:
# #!ls -a /root/.kaggle

# #!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
# if not os.path.exists('./train'):
#   !unzip -q train.zip -d train

# !ls train | wc -l

# if not os.path.exists('./test'):
#   !unzip -q test.zip -d test

# !ls test | wc -l

# Data preparation

In [0]:
train_df = pd.read_csv("./train.csv")
img_path = './train/'

#get the first 5 whale images
images = [(whale_img, whale_label) for (whale_img, whale_label) in zip(train_df.Image[:5], train_df.Id[:5])]

fig, m_axs = plt.subplots(1, len(images), figsize = (20, 10))
#show the images and label them
for ii, c_ax in enumerate(m_axs):
    c_ax.imshow(imread(os.path.join(img_path,images[ii][0])))
    c_ax.set_title(images[ii][1])

#how many photos of each whail    
#train_df.Id.value_counts()

In [0]:
# Make test directory

from distutils.dir_util import copy_tree

#create directories
if not os.path.exists('./test_folder'):
  os.mkdir('./test_folder')
  os.mkdir('./test_folder/test_images')

# copy subdirectory example
fromDirectory = img_path
toDirectory = "test_folder/test_images"

copy_tree(fromDirectory, toDirectory, verbose=0)

In [0]:
#Image size
sz = 100

# create generators
datagen=image.ImageDataGenerator(rescale=1./255, validation_split = 0.1)
test_datagen = image.ImageDataGenerator(rescale=1./255)

train_generator=datagen.flow_from_dataframe(
    dataframe=train_df, directory=img_path, 
    x_col="Image", y_col="Id", has_ext=True, seed = 42,
    class_mode="categorical", target_size=(sz,sz), batch_size=32, subset = "training")

validation_generator = datagen.flow_from_dataframe(dataframe=train_df, directory=img_path, 
    x_col="Image", y_col="Id", has_ext=True, seed = 42,
    class_mode="categorical", target_size=(sz,sz), batch_size=32, subset = "validation")

test_generator = test_datagen.flow_from_directory(directory="test_folder", 
    seed = 42, class_mode=None, target_size=(sz,sz), batch_size=1, shuffle = False)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

In [0]:
#Training metrices
from keras.metrics import top_k_categorical_accuracy

''' the function top_5_accuracy is from Peter's kernel:
    https://www.kaggle.com/pestipeti/keras-cnn-starter
'''
def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

  
chanDim = -1 # change this to 1 if you're using theano as a backend
#Create the model
  
model = Sequential()

layer = Conv2D(filters = 32, kernel_size = 4, padding = 'same', activation = 'relu', input_shape = (100, 100, 3))
model.add(layer)
model.add(Dropout(0.5))
# print(model.output_shape)
# model.add(GlobalMaxPooling2D('channels_first')) 
# print(model.output_shape)
# model.add(Flatten())
# Layer 1
# model.add(Conv2D(32, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))

# print(model.output_shape)
# # Layer 2
model.add(Dropout(0.25))
model.add(AveragePooling2D(pool_size=(3, 3), input_shape=model.output_shape))


# Layer 3
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.25))
model.add(AveragePooling2D(pool_size=(3, 3), input_shape=model.output_shape))


# Layer 4
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.25))
model.add(AveragePooling2D(pool_size=(3, 3), input_shape=model.output_shape))

# Layer 5
model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.25))
model.add(AveragePooling2D(pool_size=(3, 3), input_shape=model.output_shape))

# Layer 6
# model.add(Conv2D(512, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))
# model.add(Dropout(0.25))
# model.add(AveragePooling2D(pool_size=(3, 3), input_shape=model.output_shape))

# # Layer 7
# model.add(Conv2D(1024, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))
# model.add(Dropout(0.25))
# model.add(AveragePooling2D(pool_size=(3, 3), input_shape=model.output_shape))

# # Layer 7
# model.add(Conv2D(2048, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))
# model.add(Dropout(0.25))
# model.add(AveragePooling2D(pool_size=(3, 3), input_shape=model.output_shape))


model.add(GlobalMaxPooling2D('channels_first'))

model.add(Dense(5005, activation = 'softmax'))
# model.summary()

model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy', top_5_accuracy])

# early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5)
checkpointer = ModelCheckpoint(filepath='weights.hdf5', 
                               verbose=1, save_best_only=True)

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5)


# Train the model
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=30, callbacks = [checkpointer, early_stopping])

In [0]:
#we need to use .reset() here otherwise
#the other of predictions will be different
#then the expected
test_generator.reset()
pred = model.predict_generator(test_generator,verbose = 1,steps=7960)

In [0]:
'''This filters only the top 5 possible ids of an image'''
pred_sorted = np.argsort(-pred, axis = 1)[:,:5]
pred_sorted

In [0]:
'''
Now we generate a map of each 
index to an Id on the format 
{
0: 'w_f48451c',
1: 'w_c3d896a',
2: 'w_20df2c5',
...
}
'''
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [0]:
'''
Here we prepare pred_ids, which is a list of lists containing 
the top 5 ids by name. For example, w_13ae3d4. 
'''
from tqdm import tqdm
#create empty list
pred_ids = list()
for i,row in enumerate(tqdm(pred_sorted)):
    #create a temporary list to store the ids for a given image
    temp_list = []
    for j,value in enumerate(row):
        #for each index in pred_sorted, append the real Id in temp_list
        temp_list.append(labels[row[j]])
    #append all 5 ids for a given image to pred_ids
    #effectively creating a similar list to pred_sorted
    #but with the real ids
    pred_ids.append(temp_list)

In [0]:
'''create the final predictions by using all ids in a single string'''
final_preds = []
for i,top_5_ids in enumerate(pred_ids):
    final_preds.append(' '.join(pred_ids[i]))

In [0]:
'''delete the files on disk - otherwise the Kaggle kernel will throw an error'''
import shutil
shutil.rmtree('test_folder', ignore_errors=True)

In [0]:
# my_df_new = my_df[["c","a","b"]]

submission = pd.DataFrame({"Image": os.listdir('./test'), "Id": final_preds})
submission.to_csv("submission.csv", index = False, columns=["Image","Id"])
submission.head()

!kaggle competitions submit -c humpback-whale-identification -f submission.csv -m "Message"